# Create Op Tutorial

A simple adaptation of using the Jupyter Scala Implementation (https://github.com/alexarchambault/jupyter-scala) of these examples (https://github.com/imagej/imagej-tutorials/tree/master/create-a-new-op)

In [1]:
load.resolver("imagej.public" at "http://maven.imagej.net/content/groups/public")
load.ivy("net.imagej" % "imagej" % "2.0.0-rc-44") // use % since %% adds scala version
load.ivy("de.christophkraemer" % "rhino-script-engine" % "1.1.0")  // since it's not in the classpath with notebooks per default

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 20:00:40 CET 2015

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from imagej.public, using Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Choosing imagej.public for com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 11:44:30 CEST 2015

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPS

In [2]:
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.Cursor
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.`type`.numeric.RealType
import net.imglib2.`type`.numeric.real.DoubleType

import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin

import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.Cursor
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.`type`.numeric.RealType
import net.imglib2.`type`.numeric.real.DoubleType
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin

In [17]:
import special.wrap.obj // needed so the class is created correctly
@Plugin(`type` = classOf[Op], name = "ramp")
class Ramp extends AbstractOp {
    
    @Parameter(`type` = ItemIO.OUTPUT)
	private var rampImg: ArrayImg[DoubleType, DoubleArray] = null
    
	override def run(): Unit = {
        rampImg = ArrayImgs.doubles(256, 256);

		val c = rampImg.localizingCursor();
		val pos = new Array[Long](rampImg.numDimensions());
		while (c.hasNext()) {
			c.fwd()
			c.localize(pos)
			c.get().setReal(sum(pos))
		}
	}
    
    private def sum(pos: Array[Long]) = {
		var sum = 0.0f;
		for (p <- pos) sum += p;
		sum
	}
    /** make it accessible from outside **/
    def getOutput() = rampImg
}


defined class Ramp

In [33]:
import net.imagej.ImageJ
implicit val ij = new ImageJ()

: 

In [19]:
// Manually Register the Op as a plugin
import org.scijava.plugin.PluginInfo
val rampInfo = new PluginInfo(classOf[Ramp],
                              classOf[Op]
                             )
ij.plugin().addPlugin(rampInfo)

import org.scijava.plugin.PluginInfo
rampInfo: org.scijava.plugin.PluginInfo[net.imagej.ops.Op] = name='ramp2', priority=0.0, enabled=true, pluginType=Op

In [20]:
// call the op to make a new image
val rampImage = ij.op().run("ramp")


rampImage: java.lang.Object = ArrayImg [256x256]

In [22]:
implicit class previewDataset(curSet: net.imagej.Dataset)(implicit ij: net.imagej.ImageJ) {
    /** saves the file to a temporary directory, read it in again as a bytearray and feed it to the jupyter function **/
    def preview() = {
        val tFile = java.io.File.createTempFile("jupy",".jpg")
        curSet.
        ij.scifio().datasetIO().save(curSet,tFile.getPath())
        val imgFIS = new java.io.FileInputStream(tFile.getPath())
        val out = new Array[Byte](imgFIS.getChannel().size().toInt)
        imgFIS.read(out)
        display.jpg(out)
    }
}
import net.imglib2.`type`.numeric.RealType
implicit class dsImgPlus[T <: RealType[T], U <: net.imglib2.img.Img[T]](curImg: U)(implicit ij: net.imagej.ImageJ) {
    def toDataset() = {
        new net.imagej.DefaultDataset(ij.getContext(),net.imagej.ImgPlus.wrap(curImg))
    }
    def preview(): Unit = toDataset().preview()
}

defined class previewDataset
import net.imglib2.`type`.numeric.RealType
defined class dsImgPlus

In [47]:
import net.imglib2.RandomAccessibleInterval
val typeRamp = rampImage.asInstanceOf[net.imglib2.img.Img[DoubleType]].asInstanceOf[RandomAccessibleInterval[DoubleType]]



import net.imglib2.RandomAccessibleInterval
typeRamp: net.imglib2.RandomAccessibleInterval[net.imglib2.type.numeric.real.DoubleType] = ArrayImg [256x256]

In [57]:
import net.imglib2.converter.Converter
import net.imglib2.converter.Converters
import net.imglib2.`type`.numeric.integer.UnsignedByteType;
import net.imglib2.converter.RealUnsignedByteConverter
val c = new RealUnsignedByteConverter[DoubleType]()


import net.imglib2.converter.Converter
import net.imglib2.converter.Converters
import net.imglib2.`type`.numeric.integer.UnsignedByteType
import net.imglib2.converter.RealUnsignedByteConverter
c: net.imglib2.converter.RealUnsignedByteConverter[net.imglib2.type.numeric.real.DoubleType] = net.imglib2.converter.RealUnsignedByteConverter@1b83de35

fixie: net.imglib2.type.numeric.integer.UnsignedByteType = 230
res29_2: net.imglib2.type.numeric.integer.UnsignedByteType = 230

In [35]:
ImageJFunctions.show( view )

		RandomAccessibleInterval< FloatType > inverted = Converters.convert(
				view,
				new Converter< FloatType, FloatType >() {
					@Override
					public void convert( FloatType input, FloatType output )
					{
						output.set( -input.get() );
					}
				},
				new FloatType() );

val dsRamp = new net.imagej.DefaultDataset(ij.getContext(),net.imagej.ImgPlus.wrap(typeRamp))
dsRamp.preview()

: 

In [34]:
implicit val ijs = ij

ijs: net.imagej.ImageJ = net.imagej.ImageJ@46101740

In [36]:
ij.scifio().planeConverter().

res21: java.util.List[java.lang.String] = [ArrayDataAccess, Default, PlanarAccess]

In [39]:

ij.ui().show(ij.op().image().normalize(typeRamp))